In [1]:
import pandas as pd
from datetime import datetime
import re
import math
import matplotlib.pyplot as plt

# Импортируем библиотеку для лемматизации русских слов
import pymorphy2
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords

### Чтение готовых файлов

In [66]:
specifications = pd.read_csv('Характеристики карточек.csv')
sales = pd.read_csv('df_body.csv')
filter_words = pd.read_csv('filter_words.csv', sep=';')

# Анализ товаров категории “Боди” на маркетплейсе Wildberries

## Цели проекта:
* Выявить, какие характеристики товара генерируют наибольшую прибыль;
* Вывести список частотных ключевых слов, на основе которых будет составлено описание товара;
* Предоставить отчет в удобном виде для владельца бизнеса для самостоятельного проведения анализа.

Для получения данных о продажах заказчик предоставил в пользование сервис аналитики «Маяк». 
### Доступные данные с сервиса:
* Артикул и наименование товара;
* Кол-во продаж / упущенных продаж;
* Выручка / упущенная выручка;
* Цена (мин., сред., макс.);
* Бренд и продавец.

### Особенности работы:
Сервис позволяет скачать данные по интересующей категории, однако таблица **не содержит дату продажи**.

### Решение:
Для динамического анализа продаж было принято решение выгружать отчеты за период «месяц», присваивать месяц продажи и объединять таблицы.

### Загрузка и объединение таблиц по продажам

In [2]:
files = [['body_jan_21.xlsx', 'body_feb_21.xlsx', 'body_mar_21.xlsx', 'body_apr_21.xlsx', 'body_may_21.xlsx', 'body_jun_21.xlsx',
         'body_jul_21.xlsx', 'body_aug_21.xlsx', 'body_sep_21.xlsx' , 'body_oct_21.xlsx', 'body_nov_21.xlsx', 'body_dec_21.xlsx'],
        ['body_jan_22.xlsx', 'body_feb_22.xlsx', 'body_mar_22.xlsx', 'body_apr_22.xlsx', 'body_may_22.xlsx', 'body_jun_22.xlsx',
         'body_jul_22.xlsx', 'body_aug_22.xlsx', 'body_sep_22.xlsx' , 'body_oct_22.xlsx', 'body_nov_22.xlsx', 'body_dec_22.xlsx']]

# Создадим общую таблицу по продажам
df_body = pd.DataFrame()

for i in range(len(files)):
    for j in range(len(files[i])):
        year, mon, day = 2021+i, j+1, 1
        # Загрузим данные по продажам 2021 года
        df = pd.read_excel(files[i][j])
        # Добавим столбец с датой
        df['date'] = datetime(year, mon, day)
        # Создадим уникальный идентификатор для последующей конкатенации таблиц
        df['ID'] = df['sku_code'].astype(str) + df['date'].astype(str)
        df_body = pd.concat([df_body, df])
        
# Выгрузим итоговый файл в формате csv для дальнейшего использования в Power BI
df_body.to_csv('df_body.csv')

## Парсинг данных
Так как «Маяк» не предоставляет данные о характеристиках товара, было принято решение собрать эти данные самостоятельно.
Для этого был реализован код (см. файл "parsing.py"), где с помощью библиотеки selenium и CSS и Xpath селекторов были получены следующие данные:
* Артикул и рейтинг товара;
* Фото товара;
* Цвет;
* Состав;
* Описание;
* Декоративные элементы и др.

По согласованию с заказчиком в выборку вошли топ 1000 артикулов по показателю «количество продаж».

### Создание ссылок для парсинга

In [4]:
# Для расчета топ 1000 уникальных товаров достаточно оставить 2 столбца исходной таблицы
df = df[['sku_code', 'sales']]

# Создадим столбец со ссылкой на товар
df.sku_code = df.sku_code.astype(str)
df['urls'] = "https://www.wildberries.ru/catalog/" + df.sku_code + "/detail.aspx"

# Сгруппируем по артикулу и суммируем выручку от продаж, отсортируем по убыванию
df = df \
    .groupby('sku_code', as_index=False) \
    .agg({'sales': 'sum', 'urls': 'min'}) \
    .sort_values('sales', ascending=False) \
    .reset_index()

# Выгрузим итоговый файл в формате csv
df.urls.iloc[:1000].to_csv('urls_0_1000.csv')

## Преобразование данных полученной таблицы

In [5]:
parsing = pd.read_csv('parsing.csv')

# Переименуем столбцы таблицы в удобные для анализа в Power BI
specifications = parsing.T.rename({0: 'Артикул', 1: 'Наименование', 2: 'Рейтинг', 3: 'Состав', 4: 'Цвет', 5: 'Фото', 
6: 'Описание', 7: 'Фактура материала', 8: 'Вырез горловины', 9: 'Тип рукава', 10: 'Модель трусов', 11: 'Вид застежки',
12: 'Декоративные элементы', 13: 'Особенности модели', 14: 'Рисунок', 15: 'Покрой', 16: 'Назначение', 17: 'Тип ростовки',
18: 'Высота упаковки', 19: 'Длина упаковки', 20: 'Длина упаковки', 21: 'Уход за вещами', 22: 'Любимые герои',
23: 'Параметры модели на фото (ОГ-ОТ-ОБ)', 24: 'Размер на модели', 25: 'Рост модели на фото', 26: 'Страна производства',
27: 'Коллекция', 28: 'Комплектация', 29: 'Пол', 30: 'Сезон', 31: 'ТНВЭД'}, axis=1)

# Удалим столбец с индексами
specifications.drop(index=specifications.index[0], axis=0 , inplace=True )

# Выгрузим итоговый файл в формате csv для дальнейшего использования в Power BI
specifications.to_csv('Характеристики карточек.csv', index=False)

## Создание фильтров
Ввиду отсутствия унифицированных правил заполнения карточек товара, в  данных присутсвуют весьма разрозненные данные. Так, необходимо создать фильтры по каждой категории. Для этого отберем только уникальные слова по каждой характеристике.

Далее вручную определим ключевое слово и объединяющую категорию для каждой характеристике. Например, цвета "изумрудный" и "травяной" будут определены в одну категорию цвета "Зеленый", однако ключевое слово у каждой характеристике останется уникальным. Так как заранее мы не можем определить, какие слова могут быть использованы и наличие в этих словах ошибок и опечаток, проще всего эту работу выполнить в MS Excel. Итоговый файл: "from_to.xlsx"

Ниже представлен алгоритм сбора уникальных слов по характеристикам.

### Цвет

In [21]:
# Создадим список всех цветов, которые указали продавцы в карточках товара
color = specifications['Цвет'].to_list()
# Заменим регистр во всех словах на нижний и объединим список в одну строку
color = " ".join(map(str, color)).lower()
# Разделим строку по перечисленным символам и получим список
color = re.split('[,%; ]', color)
# Оставим в списке только уникальные значения
color = list(filter(None,set(color)))
# Удалим из списка тире, "и" и пустые значения
color = [i.lower() for i in color if i != '-' and i != 'и' and i != 'nan']
print (' \n'.join(color))

### Ключевые слова "модель трусов"

In [145]:
panties = specifications['Модель трусов']
panties = " ".join(map(str, panties)).lower()
panties = re.sub('(?<=\[)(.*?)(?=\])', 'nan', panties)
panties = re.split(' ', panties)
panties = list(filter(None,set(panties)))
panties = [i.lower() for i in panties if i != '-' and i != 'и' and i != 'nan']
print (' \n'.join(panties))

### Ключевые слова "состав"

In [44]:
sostav = specifications['Состав'].to_list()
sostav = " ".join(map(str, sostav)).lower()
sostav = re.split('[,%; ]', sostav)
for i in sostav:
    if i.isdigit():
        sostav.remove(i)
sostav = list(filter(None,set(sostav)))
sostav = [i.lower() for i in sostav if i != '-' and i != 'и' and i != 'nan']
print (' \n'.join(sostav))

### Ключевые слова "Вид застежки"

In [3]:
zastezhka = specifications['Вид застежки'].to_list()
zastezhka = [i for i in map(str, zastezhka) if i != 'nan']
for i in zastezhka:
    if '/' in i:
        zastezhka.append(i.split(r'/')[0].strip())
        zastezhka.append(i.split(r'/')[1].strip())
    elif ';' in i:
        zastezhka.append(i.split(r';')[0].strip())
        zastezhka.append(i.split(r';')[1].strip())
    elif ',' in i:
        zastezhka.append(i.split(r',')[0].strip())
        zastezhka.append(i.split(r',')[1].strip())
zastezhka = [i.lower() for i in zastezhka if '/' not in i and ';' not in i and ',' not in i]
zastezhka = list(set(zastezhka))
print (' \n'.join(zastezhka))

### Ключевые слова "Вырез горловины"

In [3]:
vyrez = specifications['Вырез горловины'].to_list()
vyrez = [i for i in map(str, vyrez) if i != 'nan']
for i in vyrez:
    if '/' in i:
        vyrez.append(i.split(r'/')[0].strip())
        vyrez.append(i.split(r'/')[1].strip())
    elif ';' in i:
        vyrez.append(i.split(r';')[0].strip())
        vyrez.append(i.split(r';')[1].strip())
    elif ',' in i:
        vyrez.append(i.split(r',')[0].strip())
        vyrez.append(i.split(r',')[1].strip())
vyrez = [i.lower() for i in vyrez if '/' not in i and ';' not in i and ',' not in i]
vyrez = list(set(vyrez))
print (' \n'.join(vyrez))

### Ключевые слова "Декоративные элементы"

In [6]:
decor = specifications['Декоративные элементы'].to_list()
decor = [i for i in map(str, decor) if i != 'nan']
for i in decor:
    if '/' in i:
        decor.append(i.split(r'/')[0].strip())
        decor.append(i.split(r'/')[1].strip())
    elif ';' in i:
        decor.append(i.split(r';')[0].strip())
        decor.append(i.split(r';')[1].strip())
    elif ',' in i:
        decor.append(i.split(r',')[0].strip())
        decor.append(i.split(r',')[1].strip())
decor = [i.lower() for i in decor if '/' not in i and ';' not in i and ',' not in i]
decor = list(set(decor))
print (' \n'.join(decor))

### Ключевые слова "Особенности модели"

In [8]:
osobennosty = specifications['Особенности модели'].to_list()
osobennosty = [i for i in map(str, osobennosty) if i != 'nan']
for i in osobennosty:
    if '/' in i:
        osobennosty.append(i.split(r'/')[0].strip())
        osobennosty.append(i.split(r'/')[1].strip())
    elif ';' in i:
        osobennosty.append(i.split(r';')[0].strip())
        osobennosty.append(i.split(r';')[1].strip())
    elif ',' in i:
        osobennosty.append(i.split(r',')[0].strip())
        osobennosty.append(i.split(r',')[1].strip())
osobennosty = [i.lower() for i in osobennosty if '/' not in i and ';' not in i and ',' not in i]
osobennosty = list(set(osobennosty))
print (' \n'.join(osobennosty))

### Ключевые слова "Рисунок"

In [6]:
risunok = specifications['Рисунок'].to_list()
risunok = [i for i in map(str, risunok) if i != 'nan']
for i in risunok:
    if '/' in i:
        risunok.append(i.split(r'/')[0].strip())
        risunok.append(i.split(r'/')[1].strip())
    elif ';' in i:
        risunok.append(i.split(r';')[0].strip())
        risunok.append(i.split(r';')[1].strip())
    elif ',' in i:
        risunok.append(i.split(r',')[0].strip())
        risunok.append(i.split(r',')[1].strip())
risunok = [i.lower() for i in risunok if '/' not in i and ';' not in i and ',' not in i]
risunok = list(set(risunok))
print (' \n'.join(risunok))

### Ключевые слова "Тип ростовки"

In [14]:
rostovka = specifications['Тип ростовки'].to_list()
rostovka = [i for i in map(str, rostovka) if i != 'nan']
for i in rostovka:
    if '/' in i:
        rostovka.append(i.split(r'/')[0].strip())
        rostovka.append(i.split(r'/')[1].strip())
    elif ';' in i:
        rostovka.append(i.split(r';')[0].strip())
        rostovka.append(i.split(r';')[1].strip())
    elif ',' in i:
        rostovka.append(i.split(r',')[0].strip())
        rostovka.append(i.split(r',')[1].strip())
rostovka = [i.lower() for i in rostovka if '/' not in i and ';' not in i and ',' not in i]
rostovka = list(set(rostovka))
print (' \n'.join(rostovka))

### Ключевые слова "Тип рукава"

In [16]:
rukava = specifications['Тип рукава'].to_list()
rukava = [i for i in map(str, rukava) if i != 'nan']
rukava = [i.lower() for i in rukava]
rukava = list(set(rukava))
print (' \n'.join(rukava))

### Ключевые слова "Фактура материала"

In [18]:
material = specifications['Фактура материала'].to_list()
material = [i for i in map(str, material) if i != 'nan']
for i in material:
    if '/' in i:
        material.append(i.split(r'/')[0].strip())
        material.append(i.split(r'/')[1].strip())
    elif ';' in i:
        material.append(i.split(r';')[0].strip())
        material.append(i.split(r';')[1].strip())
    elif ',' in i:
        material.append(i.split(r',')[0].strip())
        material.append(i.split(r',')[1].strip())
material = [i.lower() for i in material if '/' not in i and ';' not in i and ',' not in i]
material = list(set(material))
print (' \n'.join(material))

### Ключевые слова "Назначение"

In [20]:
purpose = specifications['Назначение'].to_list()
purpose = [i for i in map(str, purpose) if i != 'nan']
for i in purpose:
    if '/' in i:
        purpose.append(i.split(r'/')[0].strip())
        purpose.append(i.split(r'/')[1].strip())
    elif ';' in i:
        purpose.append(i.split(r';')[0].strip())
        purpose.append(i.split(r';')[1].strip())
    elif ',' in i:
        purpose.append(i.split(r',')[0].strip())
        purpose.append(i.split(r',')[1].strip())
purpose = [i.lower() for i in purpose if '/' not in i and ';' not in i and ',' not in i]
purpose = list(set(purpose))
print (' \n'.join(purpose))

### Ключевые слова "Коллекция"

In [22]:
collection = specifications['Коллекция'].to_list()
collection = [i for i in map(str, collection) if i != 'nan']
for i in collection:
    if '/' in i:
        collection.append(i.split(r'/')[0].strip())
        collection.append(i.split(r'/')[1].strip())
    elif ';' in i:
        collection.append(i.split(r';')[0].strip())
        collection.append(i.split(r';')[1].strip())
    elif ',' in i:
        collection.append(i.split(r',')[0].strip())
        collection.append(i.split(r',')[1].strip())
collection = [i.lower() for i in collection if '/' not in i and ';' not in i and ',' not in i]
collection = list(set(collection))
print (' \n'.join(collection))

# Эти категории используются как ключевики
### Ключевые слова "Любимые герои"

In [27]:
heros = specifications['Любимые герои'].to_list()
heros = [i for i in map(str, heros) if i != 'nan']
new_list = []
for i in heros:
    new_list.extend(i.split(' '))
heros = [i.lower() for i in new_list if '/' not in i and ';' not in i and ',' not in i]
heros = list(set(heros))
heros.sort()
print (' \n'.join(heros))

### Ключевые слова "Комплектация"

In [30]:
complection = specifications['Комплектация'].to_list()
complection = [i for i in map(str, complection) if i != 'nan']
complection = list(set(complection))
new_list = []
for i in complection:
    new_list.extend(i.split(' '))
complection = [i.lower() for i in new_list if '/' not in i and ';' not in i and ',' not in i]
complection = list(set(complection))
complection.sort()
print (' \n'.join(complection))

### Ключевые слова "Наименование"

In [4]:
names = specifications['name'].to_list()
new_list = []
for i in names:
    new_list.extend(str(i).split(' '))
names = [i.lower() for i in new_list if '/' not in i and ';' not in i and ',' not in i]
names = list(set(names))
names.sort()
print (' \n'.join(names))

## Получение списка ключевых слов
Одной из задач заказчика является выбор ключевых слов, которые можно использовать в описании товара для более качественного продвижения карточки товара и отображения карточки на топовых позициях страницы. Для этого сгенерируем список слов, которые используют продавцы, не учитывая окончания слов и словоформы.

In [15]:
# Сохраним все описания товаров в список
description = specifications['Описание'].to_list()

# Переведем список в строку
description = ' '.join(map(str, description))

# Создадим токены для каждого слова в строке с помощью nltk.tokenize
description = word_tokenize(description)

# Объявим лемматайзер для дальнейшей обработки с помощью pymorphy2
lemmatizer = pymorphy2.MorphAnalyzer()

In [16]:
# Функция для лемматизации текста, на вход принимает список токенов 
def lemmatize_text(tokens):
    # создаем переменную для хранения преобразованного текста
    text_new = ''
    # Для каждого токена в тексте
    for word in tokens:
        # С помощью лемматайзера получаем основную форму
        word = lemmatizer.parse(word)
        # Добавляем полученную лемму в переменную с преобразованным текстом
        text_new = text_new + ' ' + word[0].normal_form
    # Возвращаем преобразованный текст
    return text_new

In [ ]:
description = lemmatize_text(description)
description = description_2.split(' ')
description.to_csv('Ключевые слова.csv')

Дальнейшее преобразование происходит в **MS Excel** с помощью ручной обработки ключевых слов на предмет ошибок, создания сводных таблиц, и создания нового столбца, содержащего часть речи ключевого слова. Часть речи присваивается ключевому слову с помощью вложенной формулы =ЕСЛИ(ЕЧИСЛО(ПОИСК(A48;"ть")); "глагол"; ЕСЛИ........)))) по следующим правилам:
* если слово содержит "ть" - то это глагол;
* если слово содержить "ий", "ый", "ой" - то это прилагательное;
* остальные слова - существительные.

Итоговый файл: "Ключевые слова.csv"

# Подведение итогов


Готовый дашборд расположен по ссылке: https://clck.ru/33ZtHU

## Описание страниц дашборда
### 1. Динамика

Страница отвечает на вопросы:
* Как изменялось количество продаж в течение двух лет?
* Как изменялась сумма выручки в течение двух лет?
* Как изменялось количество товаров в течение двух лет?
* Как изменялась средняя цена в течение двух лет?
* Как изменялись все вышеперечисленные метрики в разрезе фильтров:
    * Ткань;
    * Цвет;
    * Тип рукава;
    * Трусы;
    * Вырез;
    * Декор.
    
Примечание: наименование метрик "Продажи", "Выручка", "Товары", "Средняя цена" **кликабельны**. В левом верхнем углу присутствует фильтр по годам.

### 2. Детализация

Страница отвечает на вопросы:
* Как хорошо продаются товары каждой подкатегории?
* Сколько товаров продается по каждой подкатегории?
* Какую выручку приносит каждая подкатегория?
* Какие самые популярные товары в этой подкатегории?
    
Примечание: каждая строчка в верхней таблице кликабельна. При клике на строчку фильтруется нижняя таблица по принципу "6 топовых товаров подкатегории". При отсутствии выбранной подкатегории появляется "пустой" товар, - это происходит потому, что развернутые характеристики содержат лишь 1000 (топовых) артикулов из 10000 существующих, соответственно, в этой колонке "зашиты" все остальные 9000 товаров. В левом верхнем углу присутствует фильтр по годам.

### 3. Таблица
Страница отвечает на вопросы:
* Какие данные указывал продавец в каждой карточке товара?
* Какие есть товары по выбранной подкатегории?

Примечание: в левом верхнем углу присутствует поиск по артикулу. Это опцию можно использовать, если необходимо узнать более точную информацию по выбранному товару, найденному на предыдущей странице.

### 4. Ключевые слова
Страница отвечает на вопрос:
* Какие ключевые слова в разрезе частей речи чаще всего используют продавцы WB?
Примечание: в левом верхнем углу присутствует фильтр по годам.

### 5. Идеальный боди
Страница отвечает на вопросы:
* Какие характеристики являются самыми продающими в разрезе базового и трендового продукта?
* Какие товары являются самыми показательными в разрезе подкатегорий?

Примечание: в левом верхнем углу присутствует фильтр по базовому и трендовому товару.